# Paper analytical devices (PADs) API
## About
PADs are test cards that can quickly determine whether a drug tablet contains the correct medicines. They are cheap and easy to use. They don't require power, chemicals, solvents, or any expensive instruments, so they can be deployed rapidly at large scale wherever a problem with pharmaceutical quality is suspected. More information can be found here https://padproject.nd.edu.

## The PADS projects provides an API 
The PAD project provides an API to access previously collected data from PAD cards. 

To use the API please request an API key from the team. 

This Jupyter Notebook provides examples of accessing data via the API.

In [ ]:
# If you are using Google Colaboratory then run this box and the next one
# link Collab to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Change into our folder so we can access libraries
%cd /content/drive/MyDrive/Colab\ Notebooks/user_api_example
!ls -l

In [ ]:
# PAD API database query notebook
# Chris Sweet 06/15/22
# 
# Inports and defines
# importing the helper library
import pad_helper

# set url for pad server
pad_url = 'https://pad.crc.nd.edu/'

# you need to ask Chris nicely for an API key
API_KEY = '5NWT4K7IS60WMLR3J2LV'

In [ ]:
# Load the data into pandas if successful
import pandas as pd

# lets grab some data, in the example we download all records for project "FHI2022"
my_data = pad_helper.query_pad_database("FHI2022", API_KEY)

# check if we were succesful
if my_data and 'status' in my_data:
    if my_data['status'] == 'ko':
        print("Error:", my_data['error_description'])
    else:
        # if succesful we can create a pandas table
        if 'data' in my_data:
            df = pd.DataFrame(my_data['data'])
            if 'headers' in my_data:
                df.columns = my_data['headers']
            print("Data loaded!")
            #print(my_data)
        else:
            print("Empty query!")

In [ ]:
df

## PLS example

The FHI project uses Partial least squares regression (PLS regression) to try and predict drug concentrations.
It uses image segmentation for the data and Works with this R project to generate PLSR coefficients https://github.com/sortijas/PAD . 

In this example we will download images from the ```FHI2022``` project and calculate their concentrations.

The workflow is
1. Grab some PLS coefficients. These can be generated by the R program or downloaded from the PADs website.
1. Download and loop through the images and run the image segmentation/PLS prediction code.

In [ ]:
# STAGE 1

# Lets calculate some PLS values
# first we need some coefficients
# If you need some run this to grab from server:
pls_url = 'https://pad.crc.nd.edu/neuralnetworks/pls/fhi360_concentration/1.0/pls_fhi360_conc_coefficients.csv'

# call helper function
if pad_helper.pad_download(pls_url):
    print(pls_url, "downloaded.")

In [ ]:
# STAGE 2
# Create a PLS class instance
import pad_analysis

# creat pls instance
pls_conc = pad_analysis.pls('pls_fhi360_conc_coefficients.csv')

In [ ]:
# loop over rows and get the first 10 to predict the concentration

# define temporary file
temp_file = './temp.png'

# count the rows so we can break early
row_count = 0

# for pandas we iterate with iterrows
for row in df.iterrows():
    
    # update counter
    row_count += 1

    # the data is in the second component of the row for pandas
    row_data = row[1]
    
    # get image location
    image_url = pad_url + str(row_data['processed_file_location'])
    
    # Download it to a temporary file
    if not pad_helper.pad_download(image_url, temp_file):
        continue

    # analyze it
    pls_concentration = pls_conc.quantity('./temp.png', row_data['sample_name'])
    
    # print results
    print("PAD", row_data['sample_id'], row[1]['id'], row_data['sample_name'], "PLS concentration", \
          "{:.1f}".format(pls_concentration), "%, actual", row_data['quantity'])
    
    # break once we have out 10 data points
    if row_count >= 10:
        break

## Neural Network Classification example
The PAD project uses neural networks to try and classify drugs.

In this example we will download images from the ```FHI2022``` project and predict the drug.

The workflow is
1. Grab a Neural Network. These weights/graph files can be generated by training or downloaded from the PADs website.
1. Download and loop through the images and run the NN to classify.

In [ ]:
# Lets use a Neural Network to catagorize
# first we need the NN 
# If you need one run this to grab from server:
nn_url = 'https://pad.crc.nd.edu/neuralnetworks/tf_lite/fhi360_large_454x454_image/1.0/fhi360_large_1_21_454x454_image_v1p1.tflite'

# call helper function
if pad_helper.pad_download(nn_url):
    print(nn_url, "downloaded.")

In [ ]:
# Create a NN class instance
import pad_analysis

# creat NN instance
cat_nn = pad_analysis.pad_neural_network('fhi360_large_1_21_454x454_image_v1p1.tflite')

In [ ]:
# loop over rows and get the first 10 to classify the drug

# define temporary file
temp_file = './temp.png'

# count the rows so we can break early
row_count = 0

# for pandas we iterate with iterrows
for row in df.iterrows():
    
    # update counter
    row_count += 1

    # the data is in the second component of the row for pandas
    row_data = row[1]
    
    # get image location
    image_url = pad_url + str(row_data['processed_file_location'])
    
    # Download it to a temporary file
    if not pad_helper.pad_download(image_url, temp_file):
        continue

    # analyze it
    pred_drug, conf = cat_nn.catagorize('./temp.png')
    
    # print results, last column tells us if the classification was correct
    print("PAD", row_data['sample_id'], row_data['id'], row_data['sample_name'], "NN classification", \
          pred_drug, "NN confidence", "{:.2f}".format(conf), row_data['sample_name']==pred_drug)
    
    # break once we have out 10 data points
    if row_count >= 10:
        break